In [0]:
import tensorflow as tf
import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.models import Model
from keras.utils import *
def FModel(input_shape):

    X_input = Input(input_shape)
    X = Dense(1000, kernel_initializer='normal', activation='relu')(X_input)
    X = Dense(1000, activation = 'relu', name = 'fc2')(X)
    X = Dense(500, activation = 'relu', name = 'fc3')(X)
    X = Dense(200, activation = 'relu', name = 'fc4')(X)
    X = Flatten()(X)
    X = Dense(10, activation = 'sigmoid', name = 'fc5')(X)
    model = Model(inputs = X_input, outputs = X, name = 'FModel')
    return model

In [3]:
DLModel = FModel((28,28))
DLModel.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
num_classes = 10
y_trainf = keras.utils.to_categorical(y_train, num_classes)
y_testf = keras.utils.to_categorical(y_test, num_classes)
xts = x_train.shape
yts = y_train.shape
print("x train shape is : ", xts)
print("y train shape is : ", yts)
DLModel.fit(x = x_train,y = y_trainf, epochs = 10, batch_size = 64)

x train shape is :  (60000, 28, 28)
y train shape is :  (60000,)
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
60000/60000 [==============================] - 18s 299us/step - loss: 1.6149 - acc: 0.8995
Epoch 2/10
60000/60000 [==============================] - 17s 279us/step - loss: 1.6118 - acc: 0.9000
Epoch 3/10
60000/60000 [==============================] - 17s 282us/step - loss: 1.6118 - acc: 0.9000
Epoch 4/10
60000/60000 [==============================] - 17s 281us/step - loss: 1.6118 - acc: 0.9000
Epoch 5/10
60000/60000 [==============================] - 17s 280us/step - loss: 1.6118 - acc: 0.9000
Epoch 6/10
60000/60000 [==============================] - 17s 280us/step - loss: 1.6118 - acc: 0.9000
Epoch 7/10
60000/60000 [==============================] - 17s 280us/step - loss: 1.6118 - acc: 0.9000
Epoch 8/10
60000/60000 [==============================] - 17s 279us/step - loss: 1.6118 - acc: 0.9000
Epoch 9/10
60000/60000 [==============================] - 17s 280us/ste

In [5]:
preds = DLModel.evaluate(x_test,y_testf)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(round(preds[1]*100,3)) + " %")

10000/10000 [==============================] - 1s 136us/step

Loss = 1.611809492111206
Test Accuracy = 90.0 %


In [55]:
import numpy as np
w = DLModel.get_weights()

def prunew(list1, p): #function to prune weights connections args: 2D numpy array of weight matrix of a single layer, percentage of pruning
    a = len(list1)
    N = a*(100-p)/100
    N = int(round(N)) #N = no.of elements that should remain after pruning
    final_list = [] 
    l2 = list1.flatten()
    l3 = list1.flatten()
  
    for i in range(0, N):  #a for loop which will append final_list with N highest weights of the given matrix
        max1 = 0
        ind = 0
          
        for j in range(len(l2)):  
            l2[j]
            if abs(l2[j]) > abs(max1):
              
                max1 = l2[j];
                ind = j;
        l2 = np.delete(l2,ind);           
        final_list.append(max1) 
    t = min(map(abs, final_list)) #t is the threshold ie. minimum absolute value of the weight which is allowed after pruning else prune.
    for k in range(len(l3)):
      if abs(l3[k]) < t:
        l3[k] = 0; 
      else:
        pass
    return l3 #flat list1 but it is now pruned. we will reshape it in main 

#main
we = []
wo = []
wep = []
for i in range(len(w)):
  if i%2==0:
    we.append(w[i])
  else:
    wo.append(w[i]) #every alternate weight matrices are 2D and 1D arrays hence we = even, wo is odd

for z in we:
  if len(z)>100:
    pr = prunew(z,99)
    prr = np.reshape(pr,z.shape)
    wep.append(prr) # we wont touch odd ones we only need to prune connections
  else:
    wep.append(z)
wprunedfinal = []
for i in range(5):
  wprunedfinal.append(wep[i])
  wprunedfinal.append(wo[i]) #append all the pruned and odd matrices to wprunedfinal ie. our new weight matrix

  
pmodel = DLModel
pmodel.set_weights(wprunedfinal) # make a copy of the model and set new weight matrix to tha copy
preds = pmodel.evaluate(x_test,y_testf) #evaluate loss and accuracy
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(round(preds[1]*100,3)) + " %")

10000/10000 [==============================] - 1s 138us/step

Loss = 0.38914118766784667
Test Accuracy = 90.0 %


In [56]:
 pmodel.fit(x = x_train,y = y_trainf, epochs = 5, batch_size = 64) #THIS IS VERY SURPRIZING. RETRAINING A PRUNED MODEL JUMPS UP 10%

Epoch 1/5
60000/60000 [==============================] - 18s 293us/step - loss: 0.0711 - acc: 0.9766
Epoch 2/5
60000/60000 [==============================] - 18s 292us/step - loss: 0.0399 - acc: 0.9871
Epoch 3/5
60000/60000 [==============================] - 18s 293us/step - loss: 0.0362 - acc: 0.9883
Epoch 4/5
60000/60000 [==============================] - 18s 293us/step - loss: 0.0335 - acc: 0.9892
Epoch 5/5
60000/60000 [==============================] - 18s 293us/step - loss: 0.0320 - acc: 0.9896


In [57]:
 preds = pmodel.evaluate(x_test,y_testf)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(round(preds[1]*100,3)) + " %")
del pmodel,wep, wprunedfinal, we, wo
print("deleted")

10000/10000 [==============================] - 2s 151us/step

Loss = 0.033696885677054526
Test Accuracy = 98.896 %
deleted


In [0]:
import pandas as pd
import numpy as np
def prunen2(list1, p): # list1 = 2D numpy array of weight matrix and p = percent pruning
  a = list1.shape[1]
  N = a*(100-p)/100
  N = int(round(N)) #no. of weights to keep after pruning
    
  t = np.linalg.norm(list1, axis = 0, keepdims = False) #finding l2 norm of the list1 column wise (returns an 1D array)
  t1 = t #copy of t
  t2 = np.sort(t)[::-1] #sort t in decending order
  thr = t2[N-1]#find last element of t which is at the Nth place because index starts from 0; this is our threshold
 
  for k in range(len(t1)):
    if t1[k]> thr:
      t1[k] = 1;
    else:
      t1[k] = 0; #change t1 to a 1D array of 1 and 0 if their l2 norm is greater than threshold
  pruned = list1*t1 #broadcast to set the weights of entire columns of list1 to 1 or 0 based on t1
  return pruned,t1 
w = DLModel.get_weights()
wen = []
won = []
wenp = []
wonp = []
for i in range(len(w)):
  if i%2==0:
    wen.append(w[i])
  else:
    won.append(w[i])# seperate even and odd index weght matrices.
    
for z in range(len(w)):
  if z%2==0 and z!=len(w):
    pr, index = prunen2(w[z],99) #prune
    prr = np.reshape(pr,w[z].shape)#reshape
    wenp.append(prr)
    wot = w[z+1]*index #the odd 1D weights are also set to 1 or 0 based on t1 from the function above
    wonp.append(wot)


In [103]:
wprunedfinaln = []
for i in range(5):
  wprunedfinaln.append(wenp[i])
  wprunedfinaln.append(wonp[i]) #append the pruned node weight matrix
pnmodel = DLModel
pnmodel.set_weights(wprunedfinaln) #set weights
preds = pnmodel.evaluate(x_test,y_testf)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(round(preds[1]*100,3)) + " %")

10000/10000 [==============================] - 1s 125us/step

Loss = 0.678001306438446
Test Accuracy = 73.966 %


In [104]:
pnmodel.fit(x = x_train,y = y_trainf, epochs = 4, batch_size = 64)

Epoch 1/4
60000/60000 [==============================] - 17s 281us/step - loss: 0.0716 - acc: 0.9757
Epoch 2/4
60000/60000 [==============================] - 17s 278us/step - loss: 0.0431 - acc: 0.9859
Epoch 3/4
60000/60000 [==============================] - 17s 277us/step - loss: 0.0394 - acc: 0.9872
Epoch 4/4
60000/60000 [==============================] - 17s 277us/step - loss: 0.0374 - acc: 0.9878


In [105]:
preds = pnmodel.evaluate(x_test,y_testf)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(round(preds[1]*100,3)) + " %")
del pnmodel, wprunedfinaln, wen, won, wenp,wonp
print("deleted")

10000/10000 [==============================] - 1s 144us/step

Loss = 0.03676158659607172
Test Accuracy = 98.779 %
deleted
